![https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png](https://www.pngmart.com/files/5/Spaghetti-Transparent-Background.png)

In [51]:
import pandas as pd
from math import *
import numpy as np
from scipy import interpolate

# Импорт компосовской херни
import pythoncom
from win32com.client import Dispatch, gencache

import LDefin2D
import MiscellaneousHelpers as MH



In [52]:
#  Подключим константы API Компас
kompas6_constants = gencache.EnsureModule("{75C9F5D0-B5B8-4526-8681-9903C567D2ED}", 0, 1, 0).constants
kompas6_constants_3d = gencache.EnsureModule("{2CAF168C-7961-4B90-9DA2-701419BEEFE3}", 0, 1, 0).constants

#  Подключим описание интерфейсов API5
kompas6_api5_module = gencache.EnsureModule("{0422828C-F174-495E-AC5D-D31014DBBE87}", 0, 1, 0)
kompas_object = kompas6_api5_module.KompasObject(
    Dispatch("Kompas.Application.5")._oleobj_.QueryInterface(kompas6_api5_module.KompasObject.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iKompasObject = kompas_object

#  Подключим описание интерфейсов API7
kompas_api7_module = gencache.EnsureModule("{69AC2981-37C0-4379-84FD-5DD2F3C0A520}", 0, 1, 0)
application = kompas_api7_module.IApplication(
    Dispatch("Kompas.Application.7")._oleobj_.QueryInterface(kompas_api7_module.IApplication.CLSID,
                                                             pythoncom.IID_IDispatch))
MH.iApplication = application

documents = application.Documents
#  Получим активный документ
kompas_document = application.ActiveDocument
kompas_document_2d = kompas_api7_module.IKompasDocument2D(kompas_document)
iDocument2D = kompas_object.ActiveDocument2D()

In [53]:
table_79 = pd.read_excel('../../Геодезия.xlsm', sheet_name='pandasPlan79')

table_79

,№№,X,Y,H
0,ПЗ41,1464.420000,2371.190000,129.020000
1,111,1629.315882,2375.331148,133.378038
2,112,1595.704533,2499.443419,129.468719
3,113,1540.781188,2666.469485,130.095976
4,ПЗ42,1335.770000,2585.420000,128.410000


In [54]:
table_10 = pd.read_excel('../../Геодезия.xlsm', sheet_name='pandasPlan10')

table_10

,№№,"Отсчет по дальномеру kl, м",Отсчет по гориз. Кругу,"Отметки реечн. точек Hр.т., м"
0,ПЗ41 - ПЗ42,NaN,NaN,NaN
1,1,88.0,12.033333,126.330000
2,2,111.0,78.183333,125.320000
3,3,155.0,136.400000,125.730000
4,4,132.0,144.916667,125.690000
5,5,141.0,195.450000,129.100000
6,111 - ПЗ41,NaN,NaN,NaN
7,6,94.0,125.416667,130.428038
8,7,131.0,181.350000,130.718038
9,8,128.0,231.250000,129.328038


In [55]:
def add_layer(id: int, state: int = 3, name: str = None):
    obj = iDocument2D.ksLayer(id)
    iLayerParam = kompas6_api5_module.ksLayerParam(kompas_object.GetParamStruct(kompas6_constants.ko_LayerParam))
    iLayerParam.Init()
    if name:
        iLayerParam.name = name
    iLayerParam.state = state
    iDocument2D.ksSetObjParam(obj, iLayerParam, LDefin2D.ALLPARAM)


def add_view(name: str, scale: float = 1, x=0, y=0):
    iViewParam = kompas6_api5_module.ksViewParam(kompas_object.GetParamStruct(kompas6_constants.ko_ViewParam))
    iViewParam.Init()
    iViewParam.angle = 0
    iViewParam.color = 0
    iViewParam.name = name
    iViewParam.scale_ = scale
    iViewParam.state = 3
    iViewParam.x = x
    iViewParam.y = y
    iDocument2D.ksCreateSheetView(iViewParam, 0)


def add_text(text: str, x: float, y: float, angle: float = 0, font_size: float = 2.5, color = 0):
    iParagraphParam = kompas6_api5_module.ksParagraphParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_ParagraphParam))
    iParagraphParam.Init()
    iParagraphParam.x = x
    iParagraphParam.y = y
    iParagraphParam.ang = angle
    iParagraphParam.height = 3.55
    iParagraphParam.width = 4
    iParagraphParam.hFormat = 0
    iParagraphParam.vFormat = 0
    iParagraphParam.style = 1
    iDocument2D.ksParagraph(iParagraphParam)

    iTextLineParam = kompas6_api5_module.ksTextLineParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextLineParam))
    iTextLineParam.Init()
    iTextLineParam.style = 1
    iTextItemArray = kompas_object.GetDynamicArray(LDefin2D.TEXT_ITEM_ARR)
    iTextItemParam = kompas6_api5_module.ksTextItemParam(
        kompas_object.GetParamStruct(kompas6_constants.ko_TextItemParam))
    iTextItemParam.Init()
    iTextItemParam.iSNumb = 0
    iTextItemParam.s = text
    iTextItemParam.type = 0
    iTextItemFont = kompas6_api5_module.ksTextItemFont(iTextItemParam.GetItemFont())
    iTextItemFont.Init()
    iTextItemFont.bitVector = 4096
    iTextItemFont.color = color
    iTextItemFont.fontName = "GOST type A"
    iTextItemFont.height = font_size
    iTextItemFont.ksu = 1
    iTextItemArray.ksAddArrayItem(-1, iTextItemParam)
    iTextLineParam.SetTextItemArr(iTextItemArray)

    iDocument2D.ksTextLine(iTextLineParam)
    obj = iDocument2D.ksEndObj()



In [56]:
ONE_TO_SCALE = 2000

add_view("План", 1/ONE_TO_SCALE)
add_layer(1, 3,"Основные Точки")

for i, row in table_79.iterrows():
    # X и Y перевёрнуты
    obj = iDocument2D.ksCircle(row["Y"] * 1000, row["X"] * 1000, 1000, 1)
    add_text(str(row["№№"]) + " (" + str("{:1.2f}".format(row["H"])).replace(".", ",") + ")", row["Y"] * 1000 - 15000, row["X"] * 1000 - 7000)


In [57]:
# https://stackoverflow.com/questions/7586063/how-to-calculate-the-angle-between-a-line-and-the-horizontal-axis
def angle_trunc(a):
    while a < 0.0:
        a += pi * 2
    return a

def getAngleBetweenPoints(x_orig, y_orig, x_landmark, y_landmark):
    deltaY = y_landmark - y_orig
    deltaX = x_landmark - x_orig
    return angle_trunc(atan2(deltaY, deltaX))

In [58]:
table_10

,№№,"Отсчет по дальномеру kl, м",Отсчет по гориз. Кругу,"Отметки реечн. точек Hр.т., м"
0,ПЗ41 - ПЗ42,NaN,NaN,NaN
1,1,88.0,12.033333,126.330000
2,2,111.0,78.183333,125.320000
3,3,155.0,136.400000,125.730000
4,4,132.0,144.916667,125.690000
5,5,141.0,195.450000,129.100000
6,111 - ПЗ41,NaN,NaN,NaN
7,6,94.0,125.416667,130.428038
8,7,131.0,181.350000,130.718038
9,8,128.0,231.250000,129.328038


In [59]:
add_layer(2, 3,"Побочные Точки")

extra_points = {}

for i, row in table_10.iterrows():
    if " - " in str(row["№№"]):
        origin_label, dest_label = row["№№"].split(" - ")

        if origin_label.isdigit():
            origin_label = int(origin_label)

        if dest_label.isdigit():
            dest_label = int(dest_label)

        origin = (table_79.loc[table_79['№№'] == origin_label]["Y"].values[0], table_79.loc[table_79['№№'] == origin_label]["X"].values[0])

        _dest = (table_79.loc[table_79['№№'] == dest_label]["Y"].values[0], table_79.loc[table_79['№№'] == dest_label]["X"].values[0])

        angle = np.rad2deg(getAngleBetweenPoints( origin[0], origin[1], _dest[0], _dest[1]))
        _i_am_shit_at_coding = list(origin)
        _i_am_shit_at_coding.append(table_79.loc[table_79['№№'] == origin_label]["H"].values[0])
        extra_points.update({str(origin_label): _i_am_shit_at_coding})
        continue
    else:
        if not origin:
            continue
    _d = row["Отсчет по дальномеру kl, м"]
    _alpha = row["Отсчет по гориз. Кругу"]
    xx = origin[0] + (_d * cos(np.radians(angle - _alpha)))
    yy = origin[1] + (_d * sin(np.radians(angle - _alpha)))
    # obj = iDocument2D.ksLineSeg(origin[0] * 1000, origin[1] * 1000, xx * 1000,yy * 1000, 6)

    add_text(str(row["№№"]) + " (" + str("{:1.3f}".format(row["Отметки реечн. точек Hр.т., м"])).replace(".", ",") + ")", xx * 1000 + 2000, yy * 1000 - 1000)

    obj = iDocument2D.ksCircle(xx * 1000, yy * 1000, 1000, 1)
    extra_points.update({str(row["№№"]): [xx, yy, row["Отметки реечн. точек Hр.т., м"]]})



In [60]:
interpolated_points = {}

In [61]:
# CON_P41 = "1,2,4,5,21".split(",")
# CON_1 = ["21"]
# CON_5 =["4"]

add_layer(3,2,"Точки интерполяции пашни")

CON_111 = "5,6,7,8,9,112,ПЗ41".split(",")

center_point = np.array(extra_points["111"])
for point in CON_111:
    point = np.array(extra_points[point])

    if center_point[2] > point[2]:
        _coordinate_dif = center_point[:2] - point[:2]
        xia = [center_point[2], point[2]]
        hight_dif = range(int(ceil(point[2])), int(round(center_point[2])))
    else:
        _coordinate_dif = point[:2] - center_point[:2]
        xia = [point[2], center_point[2]]
        hight_dif = range(int(ceil(center_point[2])), int(round(point[2])))

    length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
    yia = [0, length]

    _alpha = np.rad2deg(getAngleBetweenPoints( center_point[0], center_point[1], point[0], point[1]))

    x = [0, length]
    f = interpolate.interp1d(xia, yia)

    for i in hight_dif:
        _d = f(i)
        xx = center_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = center_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx  ,yy , 0)
        add_text(i, xx - 10000, yy - 8000, color=9410565)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})


In [62]:
x = [extra_points[p][0] for p in extra_points]
y = [extra_points[p][1] for p in extra_points]
centroid = (sum(x) / len(extra_points), sum(y) / len(extra_points))
obj = iDocument2D.ksPoint(*(cord * 1000 for cord in centroid), 0)

current_view = kompas_document_2d.ViewsAndLayersManager.Views.ActiveView
current_view.X = 594/2 - centroid[0] / 2
current_view.Y = 420/2 - centroid[1] / 2
current_view.Update()


True

In [63]:
def interpolate_line(first_point: str, second_point: str, interpolated_points: dict):

    _first_point = np.array(extra_points[str(first_point)])
    _second_point = np.array(extra_points[str(second_point)])

    _coordinate_dif = _first_point[:2] - _second_point[:2]

    _length = round(np.sqrt(np.sum(np.power(_coordinate_dif, 2))))

    _yia = [0, _length]
    _xia = [ _first_point[2], _second_point[2]]

    _x = [0, _length]
    _f = interpolate.interp1d(_xia, _yia)

    _alpha = np.rad2deg(getAngleBetweenPoints(_first_point[0], _first_point[1], _second_point[0], _second_point[1]))

    # if np.diff(sorted([int(round(_second_point[2])), int(ceil(_first_point[2]))])) <= 1:
    #     return

    _hight_dif_sorted = sorted([_second_point[2], _first_point[2]])

    _height_dif = range(int(ceil(_hight_dif_sorted[0])), int(_hight_dif_sorted[1]) + 1)

    print(f"Points: {first_point, second_point}\n Range: {_height_dif}\n {_hight_dif_sorted}\n")

    for i in _height_dif:
        _d = _f(i)
        xx = _first_point[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
        yy = _first_point[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
        obj = iDocument2D.ksPoint(xx, yy, 0)
        add_text(f"{i}", xx - 10000, yy - 8000, color=14417715)

        if i in interpolated_points:
            interpolated_points[i].append((xx, yy))
        else:
            interpolated_points.update({i: [(xx, yy)]})
        print(f"{i}: {xx}, {yy}")
    print("\n")

    return interpolated_points

In [64]:
# Интерполяция между 7 и 8 точками

# _seventh = np.array(extra_points["7"])
# _eighth = np.array(extra_points["8"])
#
# _coordinate_dif = _seventh[:2] - _eighth[:2]
#
# _length = round(np.sqrt(np.sum(np.power(_coordinate_dif,2))))
#
# _yia = [0, _length]
# _xia = [_seventh[2], _eighth[2]]
#
# _x = [0, _length]
# _f = interpolate.interp1d(_xia, _yia)
#
# _alpha = np.rad2deg(getAngleBetweenPoints( _seventh[0], _seventh[1], _eighth[0], _eighth[1]))
#
# _height_dif = range(int(round(_eighth[2])), int(ceil(_seventh[2])))
#
# print(_height_dif)
#
# for i in _height_dif:
#     print(i)
#     _d = _f(i)
#     xx = _seventh[0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
#     yy = _seventh[1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
#     obj = iDocument2D.ksPoint(xx  ,yy , 0)
#     add_text(f"{i}", xx - 10000, yy - 8000, color=16711680)
#
#     if i in interpolated_points:
#         interpolated_points[i].append((xx, yy))
#     else:
#         interpolated_points.update({i: [(xx, yy)]})

interpolate_line("7", "8", interpolated_points)


Points: ('7', '8')
 Range: range(130, 131)
 [129.3280382462957, 130.71803824629572]

130: 2430872.598229715, 1732721.2578401272




{130: [(2295438.999134024, 1579851.7940517827),
  (2460245.170694117, 1694029.7194678572),
  (2411439.2110369806, 1535061.3788407133),
  (2482924.2433011974, 1600178.158382683),
  (2372120.213065025, 1501460.0455749668),
  (2430872.598229715, 1732721.2578401272)],
 131: [(2319089.456250908, 1594494.6383375574),
  (2314699.7233637488, 1674765.2753948318),
  (2381028.876397862, 1746291.1130325615),
  (2435108.0891439016, 1674872.494870078),
  (2400750.148158743, 1562963.5209630902),
  (2451073.4803319066, 1608803.7930192803),
  (2373070.7454504203, 1539309.183798585)],
 132: [(2342739.9133677916, 1609137.482623332),
  (2340196.127615418, 1655653.1381607286),
  (2378632.897868289, 1697101.3108369524),
  (2409971.0075936858, 1655715.2702722987),
  (2390061.0852805055, 1590865.6630854674),
  (2419222.7173626157, 1617429.4276558778),
  (2374021.277835816, 1577158.3220222031)]}

In [65]:
add_layer(4, 3,"Кривые интерполяции пашни")

_max_num_of_points = max([len(interpolated_points[height_points]) for height_points in interpolated_points])

for height in interpolated_points:

    if len(interpolated_points[height]) <= 3:
        continue

    cent=(sum([p[0] for p in interpolated_points[height]])/len(interpolated_points[height]),sum([p[1] for p in interpolated_points[height]])/len(interpolated_points[height]))
    interpolated_points[height].sort( key=lambda p: atan2(p[1]-cent[1],p[0]-cent[0]))

    obj = iDocument2D.ksBezier(len(interpolated_points[height]) == _max_num_of_points, 1)
    for point in interpolated_points[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()

In [66]:
add_layer(5,2,"Точки интерполяции берега")

interpolated_points_shore = {}

# ПЗ 41
interpolate_line("4", "5", interpolated_points_shore)
interpolate_line("ПЗ41", "1", interpolated_points_shore)
interpolate_line("ПЗ41", "2", interpolated_points_shore)
interpolate_line("ПЗ41", "4", interpolated_points_shore)
interpolate_line("ПЗ41", "5", interpolated_points_shore)
interpolate_line("ПЗ41", "21", interpolated_points_shore)

# 112
# interpolate_line("8", "10", interpolated_points_shore)
# interpolate_line("112", "10", interpolated_points_shore)
# interpolate_line("112", "13", interpolated_points_shore)
# interpolate_line("9", "12", interpolated_points_shore)
interpolate_line("12", "13", interpolated_points_shore)
# interpolate_line("13", "1", interpolated_points_shore)
interpolate_line("1", "112", interpolated_points_shore)

# 113
interpolate_line("10", "14", interpolated_points_shore)
# interpolate_line("113", "15", interpolated_points_shore)
# interpolate_line("17", "113", interpolated_points_shore)
interpolate_line("16", "113", interpolated_points_shore)
# interpolate_line("17", "112", interpolated_points_shore)
# interpolate_line("17", "14", interpolated_points_shore)

# ПЗ 42
interpolate_line("ПЗ42", "20", interpolated_points_shore)
interpolate_line("ПЗ42", "18", interpolated_points_shore)
interpolate_line("23", "22", interpolated_points_shore)
interpolate_line("22", "21", interpolated_points_shore)
interpolate_line("20", "21", interpolated_points_shore)
interpolate_line("1", "21", interpolated_points_shore)
interpolate_line("21", "21", interpolated_points_shore)


Points: ('4', '5')
 Range: range(126, 130)
 [125.69000000000001, 129.10000000000002]

126: 2242667.996907425, 1465150.0863783914
127: 2252798.908543602, 1497931.1645688347
128: 2262929.820179779, 1530712.242759278
129: 2273060.7318159565, 1563493.3209497214


Points: ('ПЗ41', '1')
 Range: range(127, 130)
 [126.33000000000001, 129.02]

127: 2419504.1233520107, 1419336.214528435
128: 2395586.240504481, 1441654.9202074273
129: 2371668.357656951, 1463973.62588642


Points: ('ПЗ41', '2')
 Range: range(126, 130)
 [125.32000000000001, 129.02]

126: 2341440.8144548433, 1378843.4497108005
127: 2351291.5381452926, 1407180.0557668267
128: 2361142.261835742, 1435516.661822853
129: 2370992.985526191, 1463853.267878879


Points: ('ПЗ41', '4')
 Range: range(126, 130)
 [125.69000000000001, 129.02]

126: 2251784.2916476377, 1455866.010648904
127: 2291322.6056715986, 1458698.4574539026
128: 2330860.9196955594, 1461530.9042589013
129: 2370399.2337195203, 1464363.3510639


Points: ('ПЗ41', '5')
 Range: ra

{126: [(2242667.996907425, 1465150.0863783914),
  (2341440.8144548433, 1378843.4497108005),
  (2251784.2916476377, 1455866.010648904),
  (2424237.9098909814, 1341307.752404462),
  (2565199.326461905, 1280576.9740977613),
  (2466675.6436143066, 1324333.866758381),
  (2437018.7963100653, 1378675.8087245785)],
 127: [(2252798.908543602, 1497931.1645688347),
  (2419504.1233520107, 1419336.214528435),
  (2351291.5381452926, 1407180.0557668267),
  (2291322.6056715986, 1458698.4574539026),
  (2406672.3768144976, 1382073.3973036467),
  (2449191.690836691, 1445280.4253953171),
  (2573589.647432069, 1303478.6445966156),
  (2491968.857962735, 1347550.9097592079)],
 128: [(2262929.820179779, 1530712.242759278),
  (2395586.240504481, 1441654.9202074273),
  (2361142.261835742, 1435516.661822853),
  (2330860.9196955594, 1461530.9042589013),
  (2389106.843738014, 1422839.0422028312),
  (2459390.4196334844, 1502667.593587441),
  (2469583.6102051022, 1506321.8338607016),
  (2581979.968402233, 1326380.31

In [67]:
add_layer(6, 3,"Кривые интерполяции берега")

for height in interpolated_points_shore:

    if len(interpolated_points_shore[height]) < 3:
        continue

    interpolated_points_shore[height].sort()

    obj = iDocument2D.ksBezier(False, 1)
    for point in interpolated_points_shore[height]:
        dot = iDocument2D.ksPoint(*point, 0)
    iDocument2D.ksEndObj()

In [68]:
# Горизонталь вокруг ФС
add_layer(7,2,"Точки интерполяции ФС")

interpolated_points_fruit_garden = {}

_interpolation_21_23 = interpolate_line("21", "23", {})
_last_point_between_21_23 = list(_interpolation_21_23.keys())[-1]

interpolated_points_fruit_garden.update({_last_point_between_21_23: _interpolation_21_23[_last_point_between_21_23]})

_interpolation_112_17 = interpolate_line("112", "17", {})
if _last_point_between_21_23 in _interpolation_112_17:
    interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_112_17[_last_point_between_21_23])

_interpolation_14_17 = interpolate_line("14", "17", {})
interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_14_17[_last_point_between_21_23])

_interpolation_15_113 = interpolate_line("15", "113", {})
interpolated_points_fruit_garden[_last_point_between_21_23].append(*_interpolation_15_113[_last_point_between_21_23])

_interpolation_p42_18 = interpolate_line("ПЗ42", "18", {})
interpolated_points_fruit_garden[_last_point_between_21_23].insert(0, *_interpolation_p42_18[_last_point_between_21_23])


Points: ('21', '23')
 Range: range(126, 131)
 [125.00999999999999, 130.10999999999999]

126: 2468299.600446121, 1326577.3917898745
127: 2495363.171417978, 1352179.9472552747
128: 2522426.7423898354, 1377782.502720675
129: 2549490.3133616927, 1403385.0581860752
130: 2576553.88433355, 1428987.6136514756


Points: ('112', '17')
 Range: range(130, 131)
 [129.46871879258853, 130.49597558500307]

130: 2547334.9116044967, 1550001.2210275878


Points: ('14', '17')
 Range: range(129, 131)
 [128.40597558500306, 130.49597558500307]

129: 2611102.547653145, 1605182.631596719
130: 2598436.81525126, 1539405.3104002231


Points: ('15', '113')
 Range: range(129, 131)
 [128.73597558500305, 130.09597558500306]

129: 2699162.1995955533, 1598208.265075667
130: 2669332.4159293827, 1545810.1293333839


Points: ('ПЗ42', '18')
 Range: range(129, 131)
 [128.41, 130.10999999999999]

129: 2596040.7678111987, 1351211.2106585703
130: 2614042.069186112, 1377382.7541476723




In [69]:
add_layer(8, 3,"Кривые интерполяции ФС")

obj = iDocument2D.ksBezier(False, 1)
for point in interpolated_points_fruit_garden[_last_point_between_21_23]:
    dot = iDocument2D.ksPoint(*point, 0)
iDocument2D.ksEndObj()

1073756565

In [70]:
# Горизонталь за ФС
add_layer(9,2,"Точки интерполяции за ФС")

interpolated_points_behind_fruit_garden = {}

_interpolation_112_10 = interpolate_line("112", "10", {})
_last_point_between_112_10 = list(_interpolation_112_10.keys())[-1]

interpolated_points_behind_fruit_garden.update({_last_point_between_112_10: _interpolation_112_10[_last_point_between_112_10]})

_interpolation_14_17 = interpolate_line("14", "17", {})
interpolated_points_behind_fruit_garden[_last_point_between_112_10].append(*_interpolation_14_17[_last_point_between_112_10])

_interpolation_15_113 = interpolate_line("15", "113", {})
if _last_point_between_112_10 in _interpolation_15_113:
    interpolated_points_behind_fruit_garden[_last_point_between_112_10].append(*_interpolation_15_113[_last_point_between_112_10])

_interpolation_8_10 = interpolate_line("8", "10", {})
interpolated_points_behind_fruit_garden[_last_point_between_112_10].insert(0, *_interpolation_8_10[_last_point_between_112_10])


Points: ('112', '10')
 Range: range(129, 130)
 [128.65871879258853, 129.46871879258853]

129: 2524172.497892652, 1639556.3056151243


Points: ('14', '17')
 Range: range(129, 131)
 [128.40597558500306, 130.49597558500307]

129: 2611102.547653145, 1605182.631596719
130: 2598436.81525126, 1539405.3104002231


Points: ('15', '113')
 Range: range(129, 131)
 [128.73597558500305, 130.09597558500306]

129: 2699162.1995955533, 1598208.265075667
130: 2669332.4159293827, 1545810.1293333839


Points: ('8', '10')
 Range: range(129, 130)
 [128.65871879258853, 129.3280382462957]

129: 2508988.1541037885, 1689558.367944321




In [71]:
add_layer(10, 3,"Кривые интерполяции за ФС")

obj = iDocument2D.ksBezier(False, 1)
for point in interpolated_points_behind_fruit_garden[_last_point_between_112_10]:
    dot = iDocument2D.ksPoint(*point, 0)
iDocument2D.ksEndObj()

1073756580

In [72]:
# Соединить основные границы

add_layer(10, 3,"Левая граница")

_left_border = "4,5,6,7".split(sep=",")

for i in range(len(_left_border)-1):
    # print(f"{ _left_border[i]}: {extra_points[ _left_border[i]]}\t{ _left_border[i+1]}: {extra_points[ _left_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _left_border[i]][:2], *extra_points[ _left_border[i+1]][:2])], 4)

In [73]:
add_layer(11, 3,"Верхняя граница")

_top_border = "7,8,10,14,15".split(sep=",")

for i in range(len(_top_border)-1):
    # print(f"{ _top_border[i]}: {extra_points[ _top_border[i]]}\t{ _top_border[i+1]}: {extra_points[ _top_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _top_border[i]][:2], *extra_points[ _top_border[i+1]][:2])], 1)

In [74]:
add_layer(12, 3,"Граница пашни")

_p_border = "5,ПЗ41,9,112,8".split(sep=",")

for i in range(len(_p_border)-1):
    # print(f"{ _p_border[i]}: {extra_points[ _p_border[i]]}\t{ _p_border[i+1]}: {extra_points[ _p_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _p_border[i]][:2], *extra_points[ _p_border[i+1]][:2])], 4)

In [75]:
add_layer(13, 3,"Река")

_river_border = "4,2,21,20".split(sep=",")

for i in range(len(_river_border)-1):
    # print(f"{ _river_border[i]}: {extra_points[ _river_border[i]]}\t{ _river_border[i+1]}: {extra_points[ _river_border[i+1]]}")
    obj = iDocument2D.ksLineSeg(*[i * 1000 for i in (*extra_points[ _river_border[i]][:2], *extra_points[ _river_border[i+1]][:2])], 1)

# вторая сторона реки

_alpha = np.rad2deg(getAngleBetweenPoints(*extra_points["4"][:2], *extra_points["3"][:2]))

_second_shore = []
_d = 32
for i in range(len(_river_border)):
    # obj = iDocument2D.ksLineSeg(*[i * 1000 for i in tuple(map(lambda i, j: i - j, (*extra_points[ _river_border[i]][:2], *extra_points[ _river_border[i+1]][:2]), (20,20,20,20)))], 1)

    xx = extra_points[ _river_border[i]][0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
    yy = extra_points[ _river_border[i]][1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
    _second_shore.append((xx, yy))

for i in range(len(_second_shore) - 1):
    if i != len(_second_shore) - 2:
        obj = iDocument2D.ksLineSeg(*_second_shore[i], *_second_shore[i+1], 1)
    else:
        obj = iDocument2D.ksLineSeg(*_second_shore[i], *(p * 1000 for p in extra_points["19"][:2]), 1)

# Добавить подпись

_xx = (extra_points["2"][0] + extra_points["21"][0])/2 * 1000 + (_d/2 * 1000 * cos(np.radians(_alpha)))
_yy = (extra_points["2"][1] + extra_points["21"][1])/2 * 1000 + (_d/2 * 1000 * sin(np.radians(_alpha)))

add_text("р. Соть",_xx, _yy, np.rad2deg(getAngleBetweenPoints(*extra_points["2"][:2], *extra_points["21"][:2])), 5)

In [76]:
# Автодорога

add_layer(14, 3,"Автодорога")

_alpha = np.rad2deg(getAngleBetweenPoints(*extra_points["6"][:2], *extra_points["7"][:2]))

_autobahn = []
_d = 10
for i in range(len(_top_border)):

    xx = extra_points[ _top_border[i]][0] * 1000 + (_d * 1000 * cos(np.radians(_alpha)))
    yy = extra_points[ _top_border[i]][1] * 1000 + (_d * 1000 * sin(np.radians(_alpha)))
    _autobahn.append((xx, yy))

for i in range(len(_autobahn) - 1):
    obj = iDocument2D.ksLineSeg(*_autobahn[i], *_autobahn[i+1], 4)


In [77]:
from typing import List, Set, Dict, Tuple
from typing import Union

In [78]:
def endpoint_by_distance_and_angle(starting_point: Union[Tuple[float, float], List[float]], distance: float, angle: float) -> Tuple[float, float]:
    xx = starting_point[0] + (distance * cos(np.radians(angle)))
    yy = starting_point[1] + (distance * sin(np.radians(angle)))
    return xx, yy

In [79]:
# Текст пашни
add_layer(15, 3, "Текст пашни")

add_text("Пашня", *map(lambda i, j: (i + j) * 1000, extra_points["111"][:2], (-20, 45)),0, 7)

In [80]:
# Текст лес
add_layer(16, 3, "Текст лес")

add_text("Лес", *map(lambda i, j: (i + j) * 1000, extra_points["6"][:2], (-60, 0)),0, 7)

In [81]:
def m_to_mm(m: Tuple[float, float]) -> Tuple[float, float]:
    return tuple(map(lambda i: i * 1000, m))
def mm_to_m(m: Tuple[float, float]) -> Tuple[float, float]:
    return tuple(map(lambda i: i / 1000, m))

In [82]:
# https://stackoverflow.com/a/67313571
def get_intersections(x0, y0, r0, x1, y1, r1):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=sqrt((x1-x0)**2 + (y1-y0)**2)

    # non-intersecting
    if d > r0 + r1 :
        return {}
    # One circle within other
    if d < abs(r0-r1):
        return {}
    # coincident circles
    if d == 0 and r0 == r1:
        return {}
    else:
        a=(r0**2-r1**2+d**2)/(2*d)
        h=sqrt(r0**2-a**2)
        x2=x0+a*(x1-x0)/d
        y2=y0+a*(y1-y0)/d
        x3=x2+h*(y1-y0)/d
        y3=y2-h*(x1-x0)/d
        x4=x2-h*(y1-y0)/d
        y4=y2+h*(x1-x0)/d
        return x3, y3, x4, y4


In [83]:
def add_rect(starting_corner: Tuple[float, float], h: float, w: float, angle: float = 0, style: int = 1) -> int:
    i_rectangle_param = kompas6_api5_module.ksRectangleParam(kompas_object.GetParamStruct(kompas6_constants.ko_RectangleParam))
    i_rectangle_param.Init()
    i_rectangle_param.x = starting_corner[0]
    i_rectangle_param.y = starting_corner[1]
    i_rectangle_param.ang = angle
    i_rectangle_param.height = h
    i_rectangle_param.width = w
    i_rectangle_param.style = style
    return iDocument2D.ksRectangle(i_rectangle_param)

In [84]:
# Фруктовый сад
add_layer(17, 3, "Колодец")

_alpha_14_15 = np.rad2deg(getAngleBetweenPoints(*extra_points["14"][:2], *extra_points["15"][:2]))
_alpha_113_P42 = np.rad2deg(getAngleBetweenPoints(*extra_points["113"][:2], *extra_points["ПЗ42"][:2]))


# _well_angles = degrees(atan2(14.62,9.15))
#
_well_point_1 = m_to_mm(endpoint_by_distance_and_angle(extra_points["113"][:2], 40, _alpha_113_P42))
# _obj_well_point_1 = iDocument2D.ksPoint(*_well_point_1, 0)

_well_point_2 = m_to_mm(endpoint_by_distance_and_angle(extra_points["113"][:2], 58.61, _alpha_113_P42))
# _obj_well_point_2 = iDocument2D.ksPoint(*_well_point_2, 0)
#
# well_line_1 = iDocument2D.ksLineSeg(*_well_point_1, *m_to_mm(endpoint_by_distance_and_angle(mm_to_m(_well_point_1), 9.15, _alpha_113_P42 - _well_angles)), 6)

_well_point_3 = get_intersections(*_well_point_1, 9.15*1000, *_well_point_2, 14.62*1000)

# Штриховка
_well_hatch = iDocument2D.ksHatch(0, 45, 0.25, 0, 0 ,0)
_obj_well_circle = iDocument2D.ksCircle(*_well_point_3[:2], 2500 ,1)
iDocument2D.ksEndObj()

_obj_well_circle = iDocument2D.ksCircle(*_well_point_3[:2], 2500 ,1)

add_text("Колодец", *map(lambda i, j: (i + j) * 1000, mm_to_m(_well_point_3[:2]), (-10, 5)))

add_layer(18, 3, "Фруктовый сад")

# fruit_garden_points = [endpoint_by_distance_and_angle(extra_points["113"][:2], 81.5, _alpha_113_P42 - (44 + (1/60))), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 58.61, _alpha_113_P42), -8.2, _alpha_14_15), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 133.41, _alpha_113_P42), -7.81, _alpha_14_15), endpoint_by_distance_and_angle(extra_points["ПЗ42"][:2], 96.15, 0 - _alpha_113_P42 - (25 + (11/60)))]

fruit_garden_points = [tuple(extra_points["17"][:2]), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 58.61, _alpha_113_P42), -8.2, _alpha_14_15), endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 133.41, _alpha_113_P42), -7.81, _alpha_14_15), tuple(extra_points["23"][:2])]

iDocument2D.ksLineSeg(*m_to_mm(fruit_garden_points[0]), *m_to_mm(fruit_garden_points[3]), 1)
for i in range(len(fruit_garden_points)-1):
    iDocument2D.ksLineSeg(*m_to_mm(fruit_garden_points[i]), *m_to_mm(fruit_garden_points[i+1]), 1)

# _mid_fs_text = tuple(i/2 for i in (fruit_garden_points[0][0] + fruit_garden_points[1][0], fruit_garden_points[0][1] + fruit_garden_points[1][1]))
# *map(lambda i, j: (i + j) * 1000, _mid_fs_text, (-60, 0))

_alpha_fs = np.rad2deg(getAngleBetweenPoints(*fruit_garden_points[1], *fruit_garden_points[2]))

add_text("ФС", *m_to_mm(endpoint_by_distance_and_angle(fruit_garden_points[0], 15, _alpha_fs + 30)), np.rad2deg(getAngleBetweenPoints(*fruit_garden_points[0], *fruit_garden_points[1])), 5)

add_layer(19, 3, "2КЖ")

_kg2_p1 = m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 80.05, _alpha_113_P42), -8.03, _alpha_14_15))
_kg2_p2 = m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 110.23, _alpha_113_P42), -7.91, _alpha_14_15))

add_rect(_kg2_p1, -16.05*1000, (110.23-80.05)*1000, _alpha_fs)

# *tuple(i/2 for i in (_kg2_p1[0] + _kg2_p2[0], _kg2_p1[1] + _kg2_p2[1]))

add_text("2кж", _kg2_p2[0] - 2400, _kg2_p2[1] + 4000, np.rad2deg(getAngleBetweenPoints(*_kg2_p2, *_kg2_p1)),5)

In [85]:
def line_len(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    return sum(map(lambda fp, sp: (sp - fp) ** 2, p1, p2))**0.5



In [86]:
# Железная дорога
add_layer(19, 3, "Железная дорога")

_railroad_p1 = endpoint_by_distance_and_angle(extra_points["113"][:2], 12.64, _alpha_14_15)
_railroad_end = endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 162.1, _alpha_113_P42), 14.28, _alpha_14_15)

_alpha_rail = np.rad2deg(getAngleBetweenPoints(*_railroad_p1, *_railroad_end))

_railroad_p0 = endpoint_by_distance_and_angle(_railroad_p1, -70,_alpha_rail)

iDocument2D.ksLineSeg(*m_to_mm(_railroad_p0), *m_to_mm(_railroad_end), 1)

for i in range(5, int(line_len(_railroad_p0, _railroad_end))+5, 5):
    _start_point = endpoint_by_distance_and_angle(_railroad_p0, i,_alpha_rail)
    _side_point_1 = endpoint_by_distance_and_angle(_start_point, 2.5,  45 - _alpha_rail)
    _side_point_2 = endpoint_by_distance_and_angle(_start_point, -2.5, 45 - _alpha_rail)
    iDocument2D.ksLineSeg(*m_to_mm(_side_point_1), *m_to_mm(_side_point_2), 1)

In [87]:
# Условные обозначения леса, луга, фруктового сада.

def draw_meadow(point: Tuple[float, float], style: int = 2, size: float = 1000) -> Tuple[int, int]:

    line_one = iDocument2D.ksLineSeg(*point, *map(lambda i, j: (i + j), point, (0, 2.5 * size)), style)
    line_two = iDocument2D.ksLineSeg(*map(lambda i, j: (i + j), point, (1 * size, 0)), *map(lambda i, j: (i + j), point, (1 * size, 2.5 * size)), style)

    return line_one, line_two



In [88]:
# Добавляю этот пакет, что бы проще проверять находятся ли точки в полигонах
# https://stackoverflow.com/questions/52281055/point-in-polygon-using-shapely
from shapely import geometry, affinity

_plan_poly = [m_to_mm(tuple(extra_points["20"][:2])), m_to_mm(tuple(extra_points["21"][:2])), m_to_mm(tuple(extra_points["2"][:2])),
              m_to_mm(tuple(extra_points["4"][:2])), m_to_mm(tuple(extra_points["5"][:2])), m_to_mm(tuple(extra_points["6"][:2])),
              m_to_mm(tuple(extra_points["7"][:2])), m_to_mm(tuple(extra_points["8"][:2])), m_to_mm(tuple(extra_points["10"][:2])),
              m_to_mm(tuple(extra_points["14"][:2])), m_to_mm(tuple(extra_points["15"][:2])), m_to_mm(tuple(extra_points["113"][:2])),
              m_to_mm(tuple(extra_points["ПЗ42"][:2]))]

_plan_poly_line = geometry.LineString(_plan_poly)
_plan_poly = geometry.Polygon(_plan_poly_line)

_farm_poly = [m_to_mm(tuple(extra_points["ПЗ41"][:2])), m_to_mm(tuple(extra_points["5"][:2])), m_to_mm(tuple(extra_points["6"][:2])),
              m_to_mm(tuple(extra_points["7"][:2])), m_to_mm(tuple(extra_points["8"][:2])), m_to_mm(tuple(extra_points["112"][:2])),
              m_to_mm(tuple(extra_points["9"][:2]))]

_farm_poly_line = geometry.LineString(_farm_poly)
_farm_poly = geometry.Polygon(_farm_poly_line)

_fruit_garden_poly = geometry.Polygon(geometry.LineString((m_to_mm(i) for i in fruit_garden_points)))

_fruit_garden_poly = affinity.scale(_fruit_garden_poly, 0.95, 0.95)

_kg_poly = [_kg2_p1, m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 80.05, _alpha_113_P42), -8.03 - 16.05, _alpha_14_15)), m_to_mm(endpoint_by_distance_and_angle(endpoint_by_distance_and_angle(extra_points["113"][:2], 110.23, _alpha_113_P42), -7.91 -16.05, _alpha_14_15)), _kg2_p2]

_kg_poly_line = geometry.LineString(_kg_poly)
_kg_poly = geometry.Polygon(_kg_poly_line)


In [89]:
add_layer(20, 0, "Условные обозначения луга")

for i in range(int(extra_points["20"][1] - int(extra_points["20"][1]) % 5), int(extra_points["7"][1]), 20):
    for b in range(int(extra_points["4"][0] - int(extra_points["4"][1]) % 5), int(extra_points["15"][0]), 20):
        _d_point = geometry.Point(m_to_mm((b, i)))
        if _plan_poly.contains(_d_point) and not _farm_poly.contains(_d_point) and not _fruit_garden_poly.contains(_d_point):
            draw_meadow(m_to_mm((b, i)))


In [90]:
add_layer(21, 0, "Условные обозначения фруктового сада")

for i in range(int(fruit_garden_points[2][1]), int(fruit_garden_points[0][1]), 5):
    for b in range(int(fruit_garden_points[-1][0]), int(fruit_garden_points[1][0]), 5):
        _d_point = geometry.Point(m_to_mm((b, i)))
        if _fruit_garden_poly.contains(_d_point) and not _kg_poly.contains(_d_point):
            iDocument2D.ksCircle(*m_to_mm((b, i)), 1000 ,2)

In [91]:
add_layer(22, 0, "Условные обозначения леса")

for i in range(int(extra_points["4"][1] - int(extra_points["4"][1]) % 5), int(extra_points["7"][1]), 20):
    for b in range(int(extra_points["3"][0] - int(extra_points["3"][1]) % 5), int(extra_points["7"][0]), 20):
        _d_point = geometry.Point(m_to_mm((b, i)))
        if not  _plan_poly.contains(_d_point):
            iDocument2D.ksCircle(*m_to_mm((b, i)), 2500 ,2)

In [92]:
def _f_slope(x1, y1, x2, y2): # Line slope given two points:
    return (y2-y1)/(x2-x1)

def _f_angle(s1, s2):
    return degrees(atan((s2-s1)/(1+(s2*s1))))

In [93]:
# Дерево
add_layer(23, 0, "Дерево")

_a1 = radians(49 + 15/60)
_a2 = radians(_f_angle(_f_slope(*extra_points["113"][:2], *extra_points["112"][:2]), _f_slope(*extra_points["ПЗ42"][:2], *extra_points["113"][:2])) - (55 + 6/60))

s = line_len(extra_points["113"][:2], extra_points["112"][:2])

_a3 = pi - _a2 - _a1

_l1 = (s / sin(_a3)) * sin(_a1)
_l2 = (s / sin(_a3)) * sin(_a2)

_tree = endpoint_by_distance_and_angle(extra_points["113"][:2], _l1, degrees(_a2) + np.rad2deg(getAngleBetweenPoints(*extra_points["113"][:2], *extra_points["112"][:2])))

_tree_point = iDocument2D.ksPoint(*m_to_mm(_tree), 0)

from pathlib import Path

iRasterParam = kompas6_api5_module.ksRasterParam(kompas_object.GetParamStruct(kompas6_constants.ko_RasterParam))
iRasterParam.Init()
iRasterParam.embeded = True
iRasterParam.fileName = str(Path('./tree-64x64.png').absolute())
iPlacementParam = kompas6_api5_module.ksPlacementParam(kompas_object.GetParamStruct(kompas6_constants.ko_PlacementParam))
iPlacementParam.Init()
iPlacementParam.angle = 0
iPlacementParam.scale_ = 0.025

# _img_cords = tuple(map(lambda i: i * 1, iDocument2D.ksViewToSheet(*m_to_mm(_tree))[1:]))

_plan = kompas_document_2d.ViewsAndLayersManager.Views.ActiveView

# _system_view = kompas_document_2d.ViewsAndLayersManager.Views.ViewByNumber(0)
# _system_view.Current = True
# _system_view.Update()

iPlacementParam.xBase = _tree[0] / 2 - 1.847273
iPlacementParam.yBase = _tree[1] / 2
iRasterParam.SetPlace(iPlacementParam)
iDocument2D.ksInsertRaster(iRasterParam)

1073757308

In [94]:
# Рамка
add_layer(24, 0, "Рамка")

_padding = 30
_top_left_frame_corner = m_to_mm((extra_points["3"][0] - _padding, extra_points["7"][1] + _padding))
_bottom_right_frame_corner = m_to_mm((extra_points["15"][0] + _padding, extra_points["19"][1] - _padding))

_wh_dif = (-1 * (_top_left_frame_corner[1] - _bottom_right_frame_corner[1]), -1 * (_top_left_frame_corner[0] - _bottom_right_frame_corner[0]))
_max_dim = max((abs(i) for i in _wh_dif))
_wh_dif = (-1 * _max_dim, _max_dim)


# iDocument2D.ksPoint(*_top_left_frame_corner, 0)
# iDocument2D.ksPoint(*_bottom_right_frame_corner, 0)

add_rect(_top_left_frame_corner, *_wh_dif)


_padding += _padding
_top_left_frame_corner = m_to_mm((extra_points["3"][0] - _padding, extra_points["7"][1] + _padding))
_bottom_right_frame_corner = m_to_mm((extra_points["15"][0] + _padding, extra_points["19"][1] - _padding))

_wh_dif = (-1 * (_top_left_frame_corner[1] - _bottom_right_frame_corner[1]), -1 * (_top_left_frame_corner[0] - _bottom_right_frame_corner[0]))
_max_dim = max((abs(i) for i in _wh_dif))
_wh_dif = (-1 * _max_dim, _max_dim)

add_rect(_top_left_frame_corner, *_wh_dif)


1073757311

In [95]:
_main_points = [tuple(int(i // 100) / 10 for i in extra_points[point][:2]) for point in "ПЗ41,111,112,113,ПЗ42".split(sep=",")]

_min_x = min(_main_points, key=lambda x: x[0])[0]
_min_y = min(_main_points, key=lambda y: y[1])[1]

_x_list = [round(_min_x + 0.2 * i, 1) for i in range(3)]
_y_list = [round(_min_y + 0.2 * i, 1) for i in range(3)]

for x_cord in _x_list:
    for y_cord in _y_list:
        iDocument2D.ksPoint(*m_to_mm(m_to_mm((x_cord, y_cord))), 8)


In [96]:
for cord in _x_list:
    add_text(str(cord), int(cord * 1000000) + 2.5 * 2000, _bottom_right_frame_corner[1] + _padding/7 * 1000, 90,  5)
    add_text(str(cord), int(cord * 1000000) + 2.5 * 2000, _top_left_frame_corner[1]     - _padding/2.5 * 1000, 90,  5)


In [97]:
for cord in _y_list:
    add_text(str(cord), _top_left_frame_corner[0]     + _padding/7 * 1000, int(cord * 1000000) - 2.5 * 2000, 0,  5)
    add_text(str(cord), _bottom_right_frame_corner[0] + _padding/2 * 1000, int(cord * 1000000) - 2.5 * 2000, 0,  5)


In [98]:
# Добавить подпись

add_text(f"План Масштаб 1:{ONE_TO_SCALE} Вариант: ХХ", _top_left_frame_corner[0] + 10000, _bottom_right_frame_corner[1] - 40000, 0, 14)

In [99]:
if Path('../../../Watermark.png').exists():
    # Добавить картинку

    iRasterParam = kompas6_api5_module.ksRasterParam(kompas_object.GetParamStruct(kompas6_constants.ko_RasterParam))
    iRasterParam.Init()
    iRasterParam.embeded = True
    iRasterParam.fileName = str(Path('../../../Watermark.png').absolute())
    iPlacementParam = kompas6_api5_module.ksPlacementParam(kompas_object.GetParamStruct(kompas6_constants.ko_PlacementParam))
    iPlacementParam.Init()
    iPlacementParam.angle = 0
    iPlacementParam.scale_ = 0.4
    iPlacementParam.xBase = extra_points["3"][0] / 2
    iPlacementParam.yBase = extra_points["20"][1] / 2
    iRasterParam.SetPlace(iPlacementParam)
    iDocument2D.ksInsertRaster(iRasterParam)

In [100]:
kompas_document.SaveAs(str(Path(f'../test.pdf').absolute()))